##### Import Modules

In [1]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain.retrievers import BM25Retriever
from tools import CustomEmbeddings, process_file, extract_text_from_file

import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from langchain.embeddings.base import Embeddings
from PyPDF2 import PdfReader
import docx
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain.prompts import ChatPromptTemplate

import glob

In [2]:
files = glob.glob("*.pdf*")
print(files)

['38_Hannah Wolfe Resume.pdf', '38_Marketing Sales Intern_Job Description.pdf', '52_Garrett Cassells Resume.pdf', '52_Marketing Manager_Job Description.pdf']


In [8]:
job_file = files[2]
resume_file = files[0]

In [4]:
job_file

'52_Marketing Manager_Job Description.pdf'

In [5]:
# import subprocess
# subprocess.Popen([job_file],shell=True)

In [11]:
### extract job ad text
job_ad_text = extract_text_from_file(job_file)
# print(job_ad_text)

In [7]:
# print(job_ad_text)

In [7]:
### extract resume text
resume_text = extract_text_from_file(resume_file)
# print(resume_text)

In [8]:
BASE_MODEL = '/home/james/models/mistralai/Mistral-7B-Instruct-v0.3'
embeddings = CustomEmbeddings(model_name=BASE_MODEL)


/home/james/miniconda3/lib/python3.12/site-packages/transformers/quantizers/auto.py:155: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now set to True since model is quantized.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [9]:
resume_processed = process_file(resume_file, embeddings)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/home/james/miniconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'us-api.i.posthog.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ValueError: Expected each embedding in the embeddings to be a list, got ['ndarray']

In [22]:
# Function to analyze inputs and return relevant information
def analyze_inputs(job_file_path, resume_file_path, job_ad_text, resume_text):
    if job_file_path and resume_file_path:
        # Use the local Mistral model
        embeddings = CustomEmbeddings(model_name=BASE_MODEL)

        job_ad_vectorstore = process_file(job_file_path, embeddings)
        resume_vectorstore = process_file(resume_file_path, embeddings)

        if job_ad_vectorstore and resume_vectorstore:
            retriever = BM25Retriever(vector_db=job_ad_vectorstore.as_retriever())
            chain = (
                {"context": retriever, "question": passthrough}
                | PROMPT
                | ChatOllama()
                | passthrough  # Simple output parsing
            )
            response = chain.invoke("Analyze the resume based on the job description")
            return response

    return "Failed to process the files."

In [5]:


# result = analyze_inputs(uploaded_files, job_ad_text, resume_text)


In [8]:
# https://stackoverflow.com/questions/45480280/convert-scanned-pdf-to-text-python
# https://github.com/stanfordnlp/dspy/issues/835
# https://dspy-docs.vercel.app/docs/deep-dive/retrieval_models_clients/ChromadbRM


2024-06-28 16:28:37.762 
  command:

    streamlit run C:\Users\teres\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-06-28 16:28:37.764 Session state does not function when running a script without `streamlit run`
